In [ ]:
%conda install --file requirements-poetry.txt

: 

In [1]:
import polars as pl

print(
    pl.read_csv("https://j.mp/iriscsv")
    .lazy()
    .filter(pl.col("sepal_length") > 5)
    .groupby("species", maintain_order=True)
    .agg(pl.all().sum())
    .collect()
)

: 

In [1]:
import polars as pl
# df=pl.read_parquet_schema('./export.parquet')
df=pl.read_parquet('./export.parquet')
# print(df)
df.head(3)

ModuleNotFoundError: No module named 'polars'

In [1]:
import polars as pl
df=pl.read_parquet_schema('./export.parquet')
print(df)


: 

In [1]:
import time
import polars as pl

start_time = time.time()
polars_time = time.time() - start_time
df = pl.scan_csv("../../../../projects/gf-data/en.openfoodfacts.org.products.csv")
polars_time = time.time() - start_time
# Convert to Parquet
start_time = time.time()
df.sink_parquet(
    './data/openfoodfacts_products.parquet',
    compression="snappy",
    row_group_size=1_000_000
)

#df = None
polars_parquet_time = time.time() - start_time

print(f"Polars time: {polars_time:.2f} seconds")
print(f"Polars to Parquet time: {polars_parquet_time:.2f} seconds")


: 